## (DAY1-1) 사전 만들기

In [ ]:
Lexicon = dict()

with open ("en_gum-ud-train.conllu", encoding='utf8') as f:
    
    for line in f:
        line = line.strip()
        if line == '' or line.startswith("#"):
            continue

        items = line.split('\t')
        # 0:id   1:form  2:lemma   3:POS
        
        uid = items[0]
        lemma = items[2]
        pos = items[3]
        
        if uid.isdigit() and lemma.isalpha() and pos not in ('_', 'NUM', 'PUNCT', 'SYM'):
            if lemma in Lexicon:
                Lexicon[lemma] += 1
            else:
                Lexicon[lemma] = 1


In [ ]:
# 알파벳 순서로 정렬
Lexicon_list = [(x, y) for x, y in Lexicon.items()]
Lexlst_sorted = sorted(Lexicon_list)

In [ ]:
# 빈도 높은 순으로 정렬
Lexlst_sorted_freq = sorted(Lexicon_list, reverse=True, key=lambda x:x[1])
print(*Lexlst_sorted_freq[:20], sep='\n')

In [ ]:
# 단어 길이 긴 순으로 정렬
Lexlst_sorted_len = sorted(Lexicon_list, reverse=True, key=lambda x:len(x[0]))
for x, y in Lexlst_sorted_len[:100]:
    print(len(x), x, y)

In [ ]:
with open('Lexicon_gum.txt', 'w', encoding='utf8') as f:
    
    for (x, y) in Lexlst_sorted:
        f.write(x + '\t' + str(y) + '\n')

## (DAY1-2)  Anagram 찾아보기

In [ ]:
# anagram 찾아보기

Anagram = dict() # sector, escort --> ceorst (key:알파벳정렬)

with open("Lexicon_gum.txt", encoding='utf8') as f:
    for line in f:
        line = line.strip()
        word, freq = line.split('\t')
        
        key = ''.join(sorted(word))
        if key in Anagram:
            Anagram[key].append(word)
        else:
            Anagram[key] = [word]
            

In [ ]:
for (x, y) in Anagram.items():
    if len(x) >2 and len(y) >2:
        print(x, y)

## (DAY1-3) 품사 정보가 포함된 사전 구축

In [ ]:
LexPos = dict()

with open ("en_gum-ud-train.conllu", encoding='utf8') as f:
    
    for line in f:
        line = line.strip()
        if line == '' or line.startswith("#"):
            continue

        items = line.split('\t')
        # 0:id   1:form  2:lemma   3:POS
        # lemma:{NOUN:4, VERB:100}

        uid = items[0]
        lemma = items[2]
        pos = items[3]
        
        if uid.isdigit() and lemma.isalpha() and pos not in ('_', 'NUM', 'PUNCT', 'SYM'):
            if lemma in LexPos:
                if pos in LexPos[lemma]:
                    LexPos[lemma][pos] += 1
                else:
                    LexPos[lemma][pos] = 1
            else:
                LexPos[lemma] = {pos:1}    # 처음 저장 dict


In [ ]:
LexPos_sort = sorted([(x, y) for (x, y) in LexPos.items()])
LexPosLst = list()

with open('Lexicon_gum_POS.txt', 'w', encoding='utf8') as f:
    for (lemma, pos_dict) in LexPos_sort:
        total = sum([n for (p, n) in pos_dict.items()])
        pos_sort = sorted([(p,n) for (p, n) in pos_dict.items()], reverse=True, key=lambda x:x[1] )
        
        LexPosLst.append([lemma, total, pos_sort])
        
        f.write(lemma + '\t' + str(total))
        for (z1, z2) in pos_sort:
            f.write('\t'+ z1 + '\t' + str(z2))
        f.write('\n')


In [ ]:
print(len(LexPosLst))
print(*LexPosLst[4000:4100], sep='\n')

In [ ]:
# 품사 중의성이 높은 단어들

LexPosLst_sorted = sorted(LexPosLst, reverse=True, key=lambda x:len(x[2]))

for (x, y, z) in LexPosLst_sorted:
    if len(z) >2:
        print(len(z), x, y)
        print(*z)

## (DAY1-4) TREE 구조 다루기

In [ ]:
def print_text(tree):
    
    node_list = [[int(x), y[1]] for x, y in tree.items()]
    node_list.sort()
    form_list = [ x[1] for x in node_list]
    print(" ".join(form_list))
    
def print_node(node):
    
    print(node[0], ":",  node[1] + "/" + node[3],  node[4] + ':'+ node[5])
        

In [ ]:
def find_verb_cases(tree, child, cases):

    for (x, y) in child.items():
        if tree[x][3] == 'VERB':  # y = { 'obj':[1, 2] }
            if all(list(map(lambda c:c in y, cases))):
                print('\nPARENT ==> ', end='')
                print_node(tree[x]) 
                print('\tCHILDREN: ')
                for c in cases:
                    print('\t' + c + ' --> ', end='')
                    print_node(tree[y[c][0]])
                print_text(tree)

def find_node_with_how_many_child(tree, child, num_child):
    
    for (x, y) in tree.items():
        if y[3] == 'VERB' and len(y[6]) >= num_child:
            print('\nPARENT ==> ', end='')
            print_node(tree[x])
            print('CHILDREN: ', str(len(y[6])))
            for z in y[6]:
                print_node(tree[z])
            print_text(tree)
                
def find_multiple_same_relation(tree, child, rel_list):
    
    for (x, y) in child.items():  # 3:{ nmod':[1, 2, 5]}
        for z1, z2 in y.items():  
            if z1 in rel_list and len(z2) > 1:
                print('\nREL: ', z1)
                print('PARENR ==> ', end='')
                print_node(tree[x])
                print('CHILDREN: ')
                for n in z2:
                    print_node(tree[n])                
                print_text(tree)


In [ ]:
def process_tree(word_info):
    
    # word_info : [ [ID, Form, Lemma, POS, Parent, Rel] ... ]
    tree = dict()      # tree[3] = [3, sky, sky, NOUN, 5, obj, [CHILD] ]
    child = dict()     # child[3] = { 'obj':[1],  'nsubj':[2], 'obl':[6,7] }
    
    for node in word_info:
        tree[node[0]] = node + [[]] # child 정보
        child[node[0]] = dict()
    
    for node in word_info:
        
        n_id = node[0]
        p_id = node[4]
        rel = node[5]
        if p_id == '0': continue
            
        tree[p_id][6].append(n_id) # child 
        if rel in child[p_id]:            
            child[p_id][rel].append(n_id)
        else:
            child[p_id][rel]= [n_id]
            
    return tree, child
           
    
def extract_info_from_tree(word_info):
    
    tree, child = process_tree(word_info)
    
    # 용언 + relations
    #find_verb_cases(tree, child, ['nsubj', 'obj', 'obl', 'cc'])  
    
    # 주어진 rel list 중, rel child를 여러개 갖고 있는 nodes
    #find_multiple_same_relation(tree, child, ['advmod', 'obj'])
    
    # CHILD 갯수 10이상
    find_node_with_how_many_child(tree, child, 10)
            

In [ ]:
with open ("en_gum-ud-train.conllu", encoding='utf8') as f:
    
    word_info = list()
    
    for line in f:
        line = line.strip()
        
        if line.startswith("#"):   # comment
            continue
        
        if line == '' and word_info != []:
            extract_info_from_tree(word_info)
            word_info = []
            continue
            
        items = line.split('\t')
        # ID,  Form, Lemma, UPOS, XPOS,  _, Parent, Rel
        #  0    1      2     3     4    5    6      7
        if items[0].isdigit():
            word_info.append(items[0:4] + items[6:8])
        